<a href="https://colab.research.google.com/github/Parsa33033/ADM_HW3/blob/main/homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADM Homework 3

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [89]:
import requests
import os
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.request import Request, urlopen
import pandas as pd
import csv
import re
import time
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import pickle

## 1. Data collection

In [2]:
num_of_pages = 384
directory = ""
links_file = "anime.csv"

### 1.1. Get the list of animes

In [ ]:
anime = []

                               
if not os.path.isfile(links_file):
    with open(directory + links_file, "w", encoding='utf-8', newline='') as fobj:
        fobj.write("page,name,url\n")
        for page in tqdm(range(0, num_of_pages)):
            url = 'https://myanimelist.net/topanime.php?limit=' + str(page * 50)
            response = requests.get(url)

            soup = BeautifulSoup(response.text, 'html.parser')
            for tag in soup.find_all('tr'):
                links = tag.find_all('a')
                for link in links:        
                    if type(link.get('id')) == str and len(link.contents[0]) > 1:
                        x = (page + 1, link.contents[0], link.get('href'))
                        anime.append(x)
                        writer = csv.writer(fobj)
                        writer.writerow(x)

100%|██████████| 384/384 [02:53<00:00,  2.21it/s]


### 1.2. Crawl animes

In [3]:
pages_dir = directory + "pages"
page_dir = pages_dir + "/" + "page"
article = "article"

In [ ]:
if not os.path.exists(pages_dir):
    os.makedirs(pages_dir)
    
def urlEncodeNonAscii(b):
    return re.sub('[\x80-\xFF]', lambda c: '%%%02x' % ord(c.group(0)), b)

ds = pd.read_csv(directory + links_file)
i = 1
for p, u in tqdm(zip(ds.page, ds.url)):
    d = page_dir + str(p)
    if not os.path.exists(d):
        os.makedirs(d)
    d = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
    
    if not os.path.exists(d):
#         request = requests.get(u, headers={'Cache-Control': 'no-cache', 'User-Agent': 'Mozilla/5.0'})
        req = Request(u, headers={'User-Agent': 'XYZ/3.0'})
        webpage = urlopen(req, timeout=100).read()
        with open(d, "w", encoding='utf-8') as fobj:
            fobj.write(webpage)
    i += 1

### 1.3 Parse downloaded pages

In [7]:
def getText(parent):
    return ''.join(parent.find_all(text=True, recursive=False)).strip()

In [22]:
def article_parser(article_path, soup):
        anime_title = soup.title.text.strip()
        anime_type = ""
        try:
            anime_type = getText(soup.find('span', text="Type:").parent.a)
        except:
            pass
        anime_num_episodes = ""
        try:
            anime_num_episodes = getText(soup.find('span', text="Episodes:").parent)
        except:
            pass
        anime_aired = ""
        try:
            anime_aired = getText(soup.find('span', text="Aired:").parent).split(" to ")
        except:
            pass
        anime_status = ""
        try:
            anime_status = getText(soup.find('span', text="Status:").parent)
        except:
            pass
        anime_score = ""
        try:
            anime_score = getText(soup.find('span', text="Score:").parent.find_all('span', {'class': 'score-label'})[0])
        except:
            pass
        anime_users = ""
        try:
            anime_users = getText(soup.find('span', text="Members:").parent)
        except:
            pass
        anime_rank = ""
        try:
            anime_rank = getText(soup.find('span', text="Ranked:").parent)
        except:
            pass
        anime_popularity = ""
        try:
            anime_popularity = getText(soup.find('span', text="Popularity:").parent)
        except:
            pass
        anime_description = ""
        try:
            anime_description = getText(soup.find('h2', text="Synopsis").parent.parent.p)
        except:
            pass
        anime_related = []
        try: 
            anime_related = list(set(map(getText,soup.find('h2', text="Related Anime").parent.parent.tr.find_all(lambda tag:tag.name == "a" and tag.href != "")))) 
        except: 
            pass
        anime_characters = []
        try:
            anime_characters = list(filter(None,list(map(lambda tr: re.split('\n+', tr.find_all('td')[1].text.strip())[0], soup.find('h2', text="Characters & Voice Actors").find_next('div').find_all('tr')))))
        except:
            pass
        anime_voices = []
        try:
            anime_voices = list(filter(None,list(map(lambda tr: re.split('\n+', tr.find_all('td')[0].text.strip())[0], soup.find('h2', text="Characters & Voice Actors").find_next('div').find_all('tr')))))
        except:
            pass
        anime_staff = []
        try:
            anime_staff = list(map(lambda tr: re.split("\n+", tr.text.strip()), soup.find('h2', text="Staff").find_next('div').find_all('tr')))
        except:
            pass
        l = [anime_title,
                anime_type,
                anime_num_episodes,
                anime_aired,
                anime_status,
                anime_score,
                anime_users,
                anime_rank,
                anime_popularity,
                anime_description,
                anime_related,
                anime_characters,
                anime_voices,
                anime_staff]
        return l
    

In [24]:
num_of_docs = 19125
i = 1
with open('anime.tsv', 'w', encoding="utf-8", newline='') as anime:
    for p in range(1, num_of_pages + 1):
        flag = True
        if (flag): 
            article_path = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
            while os.path.isfile(article_path):
                article_path = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
                if os.path.isfile(article_path):
                    if i % 100 == 0:
                        print(p, "--->", i, "--->", article_path)
                    with open(article_path, 'rb') as html:
                        soup = BeautifulSoup(html, "html.parser")
                        l = article_parser(article_path, soup)

                        writer = csv.writer(anime, delimiter='\t')
                        writer.writerow(l)
                i += 1


2 ---> 100 ---> pages/page2/article_100.html
4 ---> 200 ---> pages/page4/article_200.html
6 ---> 300 ---> pages/page6/article_300.html
8 ---> 400 ---> pages/page8/article_400.html
10 ---> 500 ---> pages/page10/article_500.html
12 ---> 600 ---> pages/page12/article_600.html
14 ---> 700 ---> pages/page14/article_700.html
16 ---> 800 ---> pages/page16/article_800.html
18 ---> 900 ---> pages/page18/article_900.html
20 ---> 1000 ---> pages/page20/article_1000.html
22 ---> 1100 ---> pages/page22/article_1100.html
24 ---> 1200 ---> pages/page24/article_1200.html
26 ---> 1300 ---> pages/page26/article_1300.html
28 ---> 1400 ---> pages/page28/article_1400.html
30 ---> 1500 ---> pages/page30/article_1500.html
32 ---> 1600 ---> pages/page32/article_1600.html
34 ---> 1700 ---> pages/page34/article_1700.html
41 ---> 2000 ---> pages/page41/article_2000.html
43 ---> 2100 ---> pages/page43/article_2100.html
45 ---> 2200 ---> pages/page45/article_2200.html
47 ---> 2300 ---> pages/page47/article_2300.ht

329 ---> 16400 ---> pages/page329/article_16400.html
331 ---> 16500 ---> pages/page331/article_16500.html
333 ---> 16600 ---> pages/page333/article_16600.html
335 ---> 16700 ---> pages/page335/article_16700.html
337 ---> 16800 ---> pages/page337/article_16800.html
339 ---> 16900 ---> pages/page339/article_16900.html
341 ---> 17000 ---> pages/page341/article_17000.html
343 ---> 17100 ---> pages/page343/article_17100.html
345 ---> 17200 ---> pages/page345/article_17200.html
347 ---> 17300 ---> pages/page347/article_17300.html
349 ---> 17400 ---> pages/page349/article_17400.html
351 ---> 17500 ---> pages/page351/article_17500.html
353 ---> 17600 ---> pages/page353/article_17600.html
355 ---> 17700 ---> pages/page355/article_17700.html
357 ---> 17800 ---> pages/page357/article_17800.html
359 ---> 17900 ---> pages/page359/article_17900.html
361 ---> 18000 ---> pages/page361/article_18000.html
363 ---> 18100 ---> pages/page363/article_18100.html
365 ---> 18200 ---> pages/page365/article_1820

In [42]:
ds = pd.read_csv("anime.tsv", delimiter='\t')
ds.columns = ['anime_title',
                'anime_type',
                'anime_num_episodes',
                'anime_aired',
                'anime_status',
                'anime_score',
                'anime_users',
                'anime_rank',
                'anime_popularity',
                'anime_description',
                'anime_related',
                'anime_characters',
                'anime_voices',
                'anime_staff']
len(ds)

18742

In [48]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [90]:
ps = PorterStemmer()
stopwords = stopwords.words()
def preprocess(sentence):
    text_tokens = nltk.word_tokenize(sentence)
    tokens_without_sw = [ps.stem(word) for word in text_tokens if not word in stopwords and word.isalnum()]
    return tokens_without_sw

In [ ]:
# t1 = time.time()
# sentence = ds.loc[1].anime_description
# preprocess(sentence)
# t2 = time.time()
# print(t2 - t1)
index_dict = {}
for i in ds.anime_description:
    l = preprocess(i)
    print(l)
# a_file = open("data.pkl", "wb")
# pickle.dump(dictionary_data, a_file)

# a_file = open("data.pkl", "rb")
# output = pickle.load(a_file)
# print(output)

['gintoki', 'shinpachi', 'kagura', 'return', 'broke', 'member', 'yorozuya', 'team', 'live', 'edo', 'sword', 'prohibit', 'alien', 'overlord', 'conquer', 'japan', 'tri', 'thrive', 'whatev', 'work', 'get', 'hand', 'howev', 'shinpachi', 'kagura', 'still', 'paid', 'doe', 'realli', 'spend', 'cash', 'play', 'pachinko', 'meanwhil', 'gintoki', 'drunkenli', 'stagger', 'home', 'night', 'alien', 'spaceship', 'crash', 'nearbi', 'a', 'fatal', 'injur', 'crew', 'member', 'emerg', 'ship', 'give', 'gintoki', 'strang', 'devic', 'warn', 'incred', 'power', 'must', 'safeguard', 'mistak', 'alarm', 'clock', 'gintoki', 'proce', 'smash', 'devic', 'next', 'morn', 'suddenli', 'discov', 'world', 'outsid', 'apart', 'standstil', 'with', 'kagura', 'shinpachi', 'side', 'set', 'get', 'devic', 'fix', 'though', 'usual', 'noth', 'ever', 'simpl', 'yorozuya', 'team', 'fill', 'humor', 'moment', 'heartfelt', 'emot', 'fourth', 'season', 'find', 'gintoki', 'friend', 'face', 'hilari', 'misadventur', 'danger', 'crise', 'yet', 'wr

['as', 'wild', 'youth', 'elementari', 'school', 'student', 'shouya', 'ishida', 'sought', 'beat', 'boredom', 'cruelest', 'way', 'when', 'deaf', 'shouko', 'nishimiya', 'transfer', 'class', 'shouya', 'rest', 'class', 'thoughtlessli', 'bulli', 'fun', 'howev', 'mother', 'notifi', 'school', 'singl', 'blame', 'everyth', 'done', 'with', 'shouko', 'transfer', 'school', 'shouya', 'left', 'merci', 'classmat', 'he', 'heartlessli', 'ostrac', 'throughout', 'elementari', 'middl', 'school', 'teacher', 'turn', 'blind', 'eye', 'now', 'third', 'year', 'high', 'school', 'shouya', 'still', 'plagu', 'wrongdo', 'young', 'boy', 'sincer', 'regret', 'past', 'action', 'set', 'journey', 'redempt', 'meet', 'shouko', 'make', 'amend', 'tell', 'heartwarm', 'shouya', 'reunion', 'shouko', 'honest', 'attempt', 'redeem', 'continu', 'haunt', 'shadow', 'past', 'written', 'mal', 'rewrit']
['sequel', 'critic', 'acclaim', 'seri', 'begin', 'tomoya', 'okazaki', 'nagisa', 'furukawa', 'graduat', 'high', 'school', 'togeth', 'exper

['after', 'victori', 'aoba', 'jousai', 'high', 'karasuno', 'high', 'school', 'call', 'fallen', 'powerhous', 'crow', 'fli', 'final', 'reach', 'climax', 'heat', 'spring', 'tournament', 'now', 'advanc', 'nation', 'karasuno', 'team', 'defeat', 'powerhous', 'shiratorizawa', 'academi', 'karasuno', 'greatest', 'hurdl', 'adversari', 'ace', 'wakatoshi', 'ushijima', 'number', 'player', 'miyagi', 'prefectur', 'countri', 'top', 'three', 'ace', 'onli', 'strongest', 'team', 'make', 'nation', 'tournament', 'sinc', 'match', 'player', 'last', 'chanc', 'qualifi', 'nation', 'karasuno', 'use', 'everyth', 'learn', 'train', 'camp', 'prior', 'match', 'attain', 'victori', 'fill', 'restless', 'excit', 'team', 'determin', 'top', 'third', 'season', 'written', 'mal', 'rewrit']
['after', 'help', 'reviv', 'legendari', 'vampir', 'koyomi', 'araragi', 'becom', 'vampir', 'servant', 'certain', 'turn', 'back', 'human', 'regain', 'full', 'power', 'araragi', 'hunt', 'three', 'vampir', 'hunter', 'defeat', 'retriev', 'limb',

['ghostli', 'primordi', 'be', 'known', 'mushi', 'continu', 'caus', 'mysteri', 'chang', 'live', 'human', 'the', 'travel', 'mushishi', 'ginko', 'persist', 'tri', 'set', 'right', 'strang', 'unsettl', 'situat', 'encount', 'time', 'loop', 'live', 'shadow', 'telepathi', 'among', 'overt', 'effect', 'interfer', 'mushi', 'subtl', 'symptom', 'year', 'notic', 'rous', 'ginko', 'concern', 'pass', 'villag', 'villag', 'through', 'circumst', 'ginko', 'becom', 'arbit', 'determin', 'mushi', 'bless', 'curs', 'but', 'line', 'seek', 'draw', 'subject', 'some', 'patient', 'would', 'rather', 'exercis', 'new', 'power', 'utterli', 'consum', 'other', 'desper', 'strive', 'rid', 'afflict', 'fact', 'protect', 'live', 'devast', 'those', 'cross', 'path', 'mushi', 'must', 'learn', 'accept', 'seemingli', 'imposs', 'consequ', 'action', 'heal', 'wound', 'know', 'otherwis', 'risk', 'meet', 'fate', 'beyond', 'comprehens', 'written', 'mal', 'rewrit']
['the', 'fifth', 'holi', 'grail', 'war', 'fuyuki', 'citi', 'reach', 'turn'

['continu', 'peril', 'descent', 'abyss', 'riko', 'regu', 'newfound', 'friend', 'nanachi', 'reach', 'abyss', 'fifth', 'layer', 'the', 'sea', 'corps', 'upon', 'arriv', 'research', 'station', 'known', 'idofront', 'main', 'trio', 'encount', 'mysteri', 'prushka', 'alleg', 'daughter', 'bondrewd', 'lead', 'white', 'whistl', 'respons', 'nanachi', 'dark', 'past', 'despit', 'welcom', 'appear', 'idofront', 'resid', 'nanachi', 'warn', 'young', 'adventur', 'thing', 'alway', 'seem', 'with', 'rout', 'sixth', 'layer', 'shroud', 'mysteri', 'bonedrewd', 'scheme', 'await', 'sacrific', 'must', 'made', 'order', 'continu', 'journey', 'bottom', 'abyss', 'written', 'mal', 'rewrit']
['yabuki', 'joe', 'left', 'downheart', 'hopeless', 'certain', 'tragic', 'event', 'in', 'attempt', 'put', 'past', 'behind', 'joe', 'leav', 'gym', 'behind', 'begin', 'wander', 'on', 'travel', 'come', 'across', 'like', 'wolf', 'kanagushi', 'goromaki', 'gondo', 'unintent', 'fan', 'die', 'ember', 'insid', 'lead', 'put', 'wander', 'hi', 

['yotar', 'former', 'yakuza', 'member', 'fresh', 'prison', 'fixat', 'thing', 'rather', 'return', 'life', 'crime', 'young', 'aspir', 'stage', 'rakugo', 'tradit', 'japanes', 'form', 'comed', 'storytel', 'inspir', 'incarcer', 'perform', 'distinguish', 'practition', 'yakumo', 'yuurakutei', 'set', 'mind', 'meet', 'chang', 'life', 'after', 'hear', 'yotar', 'desper', 'appeal', 'mentorship', 'yakumo', 'left', 'choic', 'accept', 'first', 'apprentic', 'as', 'eagerli', 'begin', 'train', 'yotar', 'meet', 'konatsu', 'abras', 'young', 'woman', 'yakumo', 'ever', 'sinc', 'belov', 'father', 'sukeroku', 'yuurakutei', 'anoth', 'prolif', 'rakugo', 'perform', 'pass', 'away', 'through', 'hidden', 'passion', 'yotar', 'drawn', 'sukeroku', 'uniqu', 'style', 'rakugo', 'despit', 'learn', 'contrast', 'techniqu', 'upon', 'see', 'old', 'memori', 'feel', 'return', 'yakumo', 'reminisc', 'much', 'earlier', 'time', 'made', 'promis', 'greatest', 'rival', 'stori', 'set', 'past', 'present', 'depict', 'art', 'rakugo', 'rel

## 5. Algorithmic question

 In this problem, the goal is to maximize the sum of values contained in a list, where each value represents the time that the person who made the appointment wants to spend with the personal trainer, so you want to provide the personal trainer with a program that maximizes the total length of accepted appointments. 
The important condition is that the personal trainer needs a break between appointments and cannot accept two consecutive bookings. In addition, reservations are considered in the order in which they were made, so that the personal trainer cannot go back and accept past bookings, but must follow the chronological order. 

Therefore, considering the need for a break between one appointment and another and considering that past appointments cannot be accepted but it is necessary to keep to the chronological order, one solution could be to scroll through the list of bookings twice, that is once starting from the first reservation and then from the index 0, or even, and continuing with step 2 for all even indices and scroll the list a second time starting from the second reservation and then from the index 1, or odd, and proceeding with step 2 for all odd indices. 
Then add the values of the even and odd indices and compare the obtained values. The path that maximizes the sum, will be preferred. 


In [28]:
def appointments(appointments):
    l1 = []
    res1 = 0
    l2 = []
    res2 = 0
    for i in range(len(appointments)):
        if ((i % 2) == 0):
            res1 += appointments[i]
            l1.append(appointments[i])
        else:
            res2 += appointments[i]
            l2.append(appointments[i])
    if (res1 > res2):
        return l1, res1
    else:
        return l2, res2

In [29]:
mylist = [30, 40, 25, 50, 30, 20] 
print('The list is:', mylist, '\n') 
 
print('The best way to accept reservations in the given order is:', appointments(mylist)[0], 'with max duration of:', appointments(mylist)[1])

The list is: [30, 40, 25, 50, 30, 20] 

The best way to accept reservations in the given order is: [40, 50, 20] with max duration of: 110
